In [14]:
from mysklearn import myclassifiers, myevaluation, mypytable

In [15]:
# first we are going to import the dataset into a mypytable object
mytable = mypytable.MyPyTable()
mytable.load_from_file("Fraud_chop.csv")

In [16]:
# we know from my datachoping notebook what each column is and does, so for this example we are going to 
# ignore nameOrig (index of 3) nameDest (index of 6)
# we will also be dropping isFlaggedFraud (index of -1)
data = []
for row in mytable.data:
    row = []
    for i in range(len(mytable.data[0]) -1): # not using the last one
        if i != 3 and i != 6:
            row.append(mytable.data[0][i])
    data.append(row)
headers = []
for i in range(len(mytable.data[0]) -1): # not using the last one
    if i != 3 and i != 6:
        headers.append(mytable.column_names[i])
print(headers)

['step', 'type', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest', 'isFraud']


In [17]:
# we also will make x and y
X = []
y = []
for row in data:
    X.append(row[0:len(row)-1])
    y.append(row[-1])
print(len(y))
print(len(X))

387500
387500


In [20]:
import mysklearn.plot_utils as plot_utils

# it would be easier if we made a new mypytable object
# for use in graphing

new_table = mypytable.MyPyTable(data, headers)